# Aprendendo sobre Transfer Learning

Este projeto tem por finalidade aplicar os conceitos aprendidos sobre Transfer Learning. A base do estudo seguiu o projeto desenvolvido por kylemath, sendo utilizado a base de imagens desenvolvida e parte do treinamento realizado.

Desenvolver este projeto proporcionou aplicar os conceitos sobre sistemas operacionais, programação python, machine learning e deep learning. Compreender os ganhos em produtividade do uso de Transfer Learning ao aplicar a técnica e avaliar os resultados com objetos conhecidos.

## Tecnologias utilizadas

- keras
- tensorflow
- numpy
- matplotlib

## Referências

[Ensina ai](https://medium.com/ensina-ai/tutorial-transfer-learning-3972cac5e9b5)

[Adaptado de kylemath](https://colab.research.google.com/github/kylemath/ml4a-guides/blob/master/notebooks/transfer-learning.ipynb#scrollTo=3p-OjhDPYoZm)

Dataset: [caltech edu](https://data.caltech.edu/records/mzrjq-6wc02)

In [2]:
%matplotlib inline

import os

#if using Theano with GPU
os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [3]:
!echo "Extraindo pasta com a base das imagens."
!tar -xzf 101_ObjectCategories.tar.gz
!rm 101_ObjectCategories.tar.gz
!ls

Extraindo pasta com a base das imagens.
101_ObjectCategories  sample_data


In [4]:
root = '101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

['101_ObjectCategories/kangaroo', '101_ObjectCategories/butterfly', '101_ObjectCategories/minaret', '101_ObjectCategories/crocodile_head', '101_ObjectCategories/ant', '101_ObjectCategories/metronome', '101_ObjectCategories/ewer', '101_ObjectCategories/lamp', '101_ObjectCategories/stop_sign', '101_ObjectCategories/car_side', '101_ObjectCategories/bass', '101_ObjectCategories/binocular', '101_ObjectCategories/elephant', '101_ObjectCategories/joshua_tree', '101_ObjectCategories/trilobite', '101_ObjectCategories/ketch', '101_ObjectCategories/scorpion', '101_ObjectCategories/grand_piano', '101_ObjectCategories/windsor_chair', '101_ObjectCategories/buddha', '101_ObjectCategories/saxophone', '101_ObjectCategories/revolver', '101_ObjectCategories/lobster', '101_ObjectCategories/laptop', '101_ObjectCategories/hawksbill', '101_ObjectCategories/platypus', '101_ObjectCategories/garfield', '101_ObjectCategories/pigeon', '101_ObjectCategories/crab', '101_ObjectCategories/strawberry', '101_ObjectCate

In [5]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [6]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

In [7]:
random.shuffle(data)

In [8]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

In [9]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

[19, 92, 39, 75, 85, 13, 69, 66, 86, 40, 65, 25, 43, 15, 52, 52, 49, 86, 68, 42, 83, 10, 78, 3, 2, 86, 57, 48, 14, 87, 40, 31, 61, 68, 46, 86, 94, 40, 18, 83, 35, 87, 71, 85, 92, 44, 85, 15, 94, 6, 24, 55, 69, 25, 37, 47, 66, 18, 47, 95, 83, 47, 44, 71, 43, 85, 39, 87, 18, 20, 58, 12, 85, 8, 3, 59, 57, 59, 89, 14, 85, 2, 32, 2, 38, 87, 72, 45, 66, 16, 21, 62, 53, 10, 23, 81, 79, 15, 33, 63, 41, 6, 96, 75, 53, 93, 32, 50, 6, 66, 49, 83, 3, 9, 1, 85, 39, 64, 78, 86, 68, 26, 83, 79, 11, 61, 53, 13, 34, 71, 6, 52, 37, 71, 17, 93, 9, 9, 13, 37, 30, 17, 24, 75, 94, 86, 15, 25, 6, 14, 57, 78, 12, 90, 23, 16, 41, 89, 88, 39, 83, 40, 38, 35, 74, 94, 21, 16, 61, 39, 6, 6, 51, 21, 36, 51, 86, 1, 18, 87, 36, 58, 37, 86, 11, 64, 95, 96, 26, 83, 40, 58, 68, 41, 63, 42, 15, 2, 39, 64, 83, 1, 78, 43, 35, 15, 56, 61, 46, 40, 94, 1, 16, 11, 94, 95, 50, 90, 68, 75, 37, 75, 6, 34, 50, 84, 37, 42, 24, 84, 92, 36, 17, 9, 56, 37, 14, 92, 16, 57, 63, 15, 75, 95, 11, 77, 74, 66, 13, 6, 95, 50, 13, 14, 32, 30, 

In [10]:
# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

(932, 97)


In [11]:
# summary
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)


finished loading 6209 images from 97 categories
train / validation / test split: 4346, 931, 932
training data shape:  (4346, 224, 224, 3)
training labels shape:  (4346, 97)


In [ ]:
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(16,4))
plt.imshow(concat_image)

Aplicando Transfer Learning com base no treinamento VGG-16.

In [13]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

553467096/553467096 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [14]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)

In [15]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
history2 = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))

Epoch 1/10
34/34 [==============================] - 1049s 31s/step - loss: 4.7842 - accuracy: 0.1240 - val_loss: 3.3529 - val_accuracy: 0.2900
Epoch 2/10
34/34 [==============================] - 1042s 31s/step - loss: 2.7964 - accuracy: 0.3909 - val_loss: 2.5834 - val_accuracy: 0.4447
Epoch 3/10
34/34 [==============================] - 1041s 31s/step - loss: 2.2847 - accuracy: 0.5067 - val_loss: 2.2324 - val_accuracy: 0.5027
Epoch 4/10
34/34 [==============================] - 1042s 31s/step - loss: 1.9703 - accuracy: 0.5771 - val_loss: 2.0183 - val_accuracy: 0.5532
Epoch 5/10
34/34 [==============================] - 1042s 31s/step - loss: 1.7690 - accuracy: 0.6173 - val_loss: 1.9068 - val_accuracy: 0.5811
Epoch 6/10
34/34 [==============================] - 1043s 31s/step - loss: 1.5941 - accuracy: 0.6505 - val_loss: 1.8037 - val_accuracy: 0.5983
Epoch 7/10
34/34 [==============================] - 1043s 31s/step - loss: 1.4679 - accuracy: 0.6802 - val_loss: 1.6400 - val_accuracy: 0.6412

In [23]:
#Validação do treinamento
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 1.4933192729949951
Test accuracy: 0.6652360558509827


In [24]:
img, x = get_image('101_ObjectCategories/airplanes/image_0003.jpg')
probabilities = model_new.predict([x])

1/1 [==============================] - 1s 565ms/step


In [26]:
#Probabilidade de a imagem ser um das categorias
probabilities

array([[3.44915728e-11, 2.57987082e-01, 1.03953258e-20, 3.74713717e-16,
        2.80400161e-14, 2.45713814e-08, 6.02143728e-14, 6.49768079e-16,
        2.16008880e-06, 2.91468322e-10, 7.06454140e-09, 1.17143216e-04,
        1.04748939e-10, 4.70629844e-15, 2.97361703e-17, 1.71529422e-07,
        5.21905097e-09, 1.74954005e-06, 1.23597519e-03, 1.36041888e-06,
        1.64220790e-11, 6.05337003e-10, 4.69719241e-09, 5.01263060e-07,
        1.50354325e-08, 9.37369177e-18, 1.10231241e-10, 1.67742163e-13,
        5.24197054e-15, 4.19510812e-15, 2.79329987e-21, 1.06889281e-14,
        7.29431631e-05, 4.36498522e-04, 3.08127142e-08, 5.83553442e-13,
        3.64376262e-09, 1.81844609e-10, 2.21137721e-02, 1.55086097e-10,
        4.62383343e-12, 4.09941858e-09, 4.84706168e-14, 1.33936977e-04,
        1.01731957e-22, 6.45534237e-10, 3.24011774e-17, 5.42071203e-14,
        1.03902228e-14, 7.67111166e-11, 5.45231998e-17, 2.10775975e-15,
        1.25090774e-13, 1.00703321e-04, 3.64595541e-11, 1.105163